# Literature

There are a set of popular graph network architectures implemented already in `kgcnn` . They can be found in `kgcnn.literature` . Most models are set up in the functional ``keras`` API. Information on hyperparameters, training and benchmarking can be found below.

Following models are implemented at the moment in [kgcnn](https://github.com/aimat-lab/gcnn_keras/tree/master/kgcnn/literature)=2.2.2

* **[GCN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GCN)**: [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907) by Kipf et al. (2016)
* **[Schnet](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/Schnet)**: [SchNet – A deep learning architecture for molecules and materials ](https://aip.scitation.org/doi/10.1063/1.5019779) by Schütt et al. (2017)
* **[GAT](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GAT)**: [Graph Attention Networks](https://arxiv.org/abs/1710.10903) by Veličković et al. (2018)
* **[GraphSAGE](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GraphSAGE)**: [Inductive Representation Learning on Large Graphs](http://arxiv.org/abs/1706.02216) by Hamilton et al. (2017)
* **[DimeNetPP](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/DimeNetPP)**: [Fast and Uncertainty-Aware Directional Message Passing for Non-Equilibrium Molecules](https://arxiv.org/abs/2011.14115) by Klicpera et al. (2020)
* **[GNNExplainer](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GNNExplain)**: [GNNExplainer: Generating Explanations for Graph Neural Networks](https://arxiv.org/abs/1903.03894) by Ying et al. (2019)
* **[AttentiveFP](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/AttentiveFP)**: [Pushing the Boundaries of Molecular Representation for Drug Discovery with the Graph Attention Mechanism](https://pubs.acs.org/doi/10.1021/acs.jmedchem.9b00959) by Xiong et al. (2019)
* **[INorp](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/INorp)**: [Interaction Networks for Learning about Objects,Relations and Physics](https://arxiv.org/abs/1612.00222) by Battaglia et al. (2016)
* **[Megnet](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/Megnet)**: [Graph Networks as a Universal Machine Learning Framework for Molecules and Crystals](https://doi.org/10.1021/acs.chemmater.9b01294) by Chen et al. (2019)
* **[NMPN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/NMPN)**: [Neural Message Passing for Quantum Chemistry](http://arxiv.org/abs/1704.01212) by Gilmer et al. (2017)
* **[Unet](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/Unet)**: [Graph U-Nets](http://proceedings.mlr.press/v97/gao19a/gao19a.pdf) by H. Gao and S. Ji (2019)
* **[GATv2](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GATv2)**: [How Attentive are Graph Attention Networks?](https://arxiv.org/abs/2105.14491) by Brody et al. (2021)
* **[GIN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GIN)**: [How Powerful are Graph Neural Networks?](https://arxiv.org/abs/1810.00826) by Xu et al. (2019)
* **[PAiNN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/PAiNN)**: [Equivariant message passing for the prediction of tensorial properties and molecular spectra](https://arxiv.org/pdf/2102.03150.pdf) by Schütt et al. (2020)
* **[DMPNN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/DMPNN)**: [Analyzing Learned Molecular Representations for Property Prediction](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.9b00237) by Yang et al. (2019)
* **[HamNet](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/HamNet)**: [HamNet: Conformation-Guided Molecular Representation with Hamiltonian Neural Networks](https://arxiv.org/abs/2105.03688) by Li et al. (2021)
* **[CGCNN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/CGCNN.py)**: [Crystal Graph Convolutional Neural Networks for an Accurate and Interpretable Prediction of Material Properties](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.145301) by Xie et al. (2018)
* **[CMPNN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/CMPNN)**: [Communicative Representation Learning on Attributed Molecular Graphs](https://www.ijcai.org/proceedings/2020/0392.pdf) by Song et al. (2020)
* **[EGNN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/EGNN)**: [E(n) Equivariant Graph Neural Networks](https://arxiv.org/abs/2102.09844) by Satorras et al. (2021)
* **[MAT](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/MAT)**: [Molecule Attention Transformer](https://arxiv.org/abs/2002.08264) by Maziarka et al. (2020)
* **[MXMNet](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/MXMNet)**: [Molecular Mechanics-Driven Graph Neural Network with Multiplex Graph for Molecular Structures](https://arxiv.org/abs/2011.07457) by Zhang et al. (2020)
* **[RGCN](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/RGCN)**: [Modeling Relational Data with Graph Convolutional Networks](https://arxiv.org/abs/1703.06103) by Schlichtkrull et al. (2017)
* **[GNNFilm](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/GNNFilm)**: [GNN-FiLM: Graph Neural Networks with Feature-wise Linear Modulation](https://arxiv.org/abs/1906.12192) by Marc Brockschmidt (2020)
* **[HDNNP2nd](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/HDNNP2nd)**: [Atom-centered symmetry functions for constructing high-dimensional neural network potentials](https://aip.scitation.org/doi/abs/10.1063/1.3553717) by Jörg Behler (2011)
* **[HDNNP4th](https://github.com/aimat-lab/gcnn_keras/blob/master/kgcnn/literature/HDNNP4th)**: [A fourth-generation high-dimensional neural network potential with accurate electrostatics including non-local charge transfer](https://www.nature.com/articles/s41467-020-20427-2) by Ko et al. (2021)

## Training Scripts

Currently there are training scripts *train_citation.py*, *train_crystal.py*, *train_force.py*, *train_moleculenet.py*, *train_qm.py*, *train_tudataset.py* and *train_visual_graph_dataset.py* for different dataset types.

> **NOTE**: They are quite integrated with ``kgcnn`` models and datasets which is why a custom training script can be favorable for models not in `kgcnn.literature`.

Training scripts can be started with:


```bash
python3 train_citation.py --dataset CoraLuDataset --model GAT --hyper hyper/hyper_esol.py
python3 train_moleculenet.py --dataset ESOLDataset --model GIN --hyper hyper/hyper_esol.py
python3 train_tudataset.py --dataset PROTEINSDataset --model GIN --hyper hyper/hyper_proteins.py
```

Where `hyper_esol.py` stores hyperparameter and must be in the same folder or a path to a `.py`. 

In principle, training can be fully configured with a serialized hyper parameter file as 'hyper.json' or 'hyper.yaml'.
If a pyhton file 'hyper.py' is used then `hyper = {...}` must be set in the python script, in which case the items do not necessarily need to be in serailized form.

In [1]:
hyper = {
    "info":{ 
        # General information for training run
        "kgcnn_version": "2.2.2", # Version 
        "postfix": "" # Postfix for output folder.
    },
    "model": { 
        # Model specific parameter, see kgcnn.literature.
    },
    "data": { 
        # Dataset specific parameters.
    },
    "training": {
        "fit": { 
            # serialized keras fit arguments.
        },
        "compile": { 
            # serialized keras compile arguments.
        },
        "cross_validation": {
            # serialized parameters for cross-validation.  
        },
        "scaler": {
            # Label scaler, only add when training for regression.
        }
    }
}

#### Data hyperparameter

The kwargs for the dataset are not fully identical and vary a little depending on the datset. However, the most common are listed below.

In [2]:
hyper.update({
    "data":{
        "dataset": {
            "class_name": "QM9Dataset", # Name of the dataset
            "module_name": "kgcnn.data.datasets.QM9Dataset",
            
            # Config like filepath etc., leave empty for pre-defined datasets
            "config": {}, 
            
            # Methods to run on dataset, i.e. the list of graphs
            "methods": [
                {"prepare_data": {}}, # Used for cache and pre-compute data, leave out for pre-defined datasets
                {"read_in_memory": {}}, # Used for reading into memory, leave out for pre-defined datasets
                
                # Example method to run over each graph in the list using `map_list` method.
                # The string 'set_range' refers to a preprocessor. Legacy short access to graph preprocessors.
                {"map_list": {"method": "set_range", "max_distance": 4, "max_neighbours": 30}}
            ]
        },
        
        # Other optinal entries (depends on the training script)
        "data_unit": "mol/L",
    }
})

#### Model hyperparameter

The model parameters can be reviewed from the default values in ``kgcnn.literature``. Mostly model input and output has to be matched depending on the data representation. That is type of input and its shape. An input-type checker can be used from `kgcnn.data.base.MemoryGraphDataset`, which has `assert_valid_model_input`. In ``inputs`` a list of kwargs must be given, which are each unpacked in the corresponding ``tf.keras.layers.Input``. The order matters and is model dependent.

Moreover, naming of the model input is used to link the tensor properties of the dataset with the model input. The output dimension of either node or graph embedding can be set for most models with the "output_mlp" argument.

In [3]:
hyper.update({
    "model":{
        "module_name": "kgcnn.literature.GCN", 
        "class_name": "make_model",
        "config":{
            # Name of the inputs must match graph properties in the dataset
            "inputs": [
                {"shape": [None, 100], "name": "node_attributes", "dtype": "float32", "ragged": True},
                {"shape": [None, 1], "name": "edge_weights", "dtype": "float32", "ragged": True},
                {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True}],
            # More model specific kwargs, like:
            "depth": 5,
            # Output part defining model output
            "output_embedding": "graph",
            "output_mlp": {"use_bias": [True, True, False], "units": [140, 70, 70],
                           "activation": ["relu", "relu", "softmax"]}
        }
    }
})

#### Training hyperparameter

The kwargs for training simply sets arguments for ``model.compile(**kwargs)`` and ``model.fit(**kwargs)`` that matches keras arguments as well as for the k-fold split from scikit-learn. The kwargs are expected to be fully serialized, if the hyper parameters are supposed to be saved to json.

In [4]:
import tensorflow as tf
hyper.update({
    "training":{
        # Cross-validation of the data
        "cross_validation": {
            "class_name": "KFold",
            "config": {"n_splits": 5, "random_state": 42, "shuffle": True}
        },
        # Standard scaler for regression targets
        "scaler": {
            "class_name": "StandardScaler",
            "module_name": "kgcnn.data.transform.scaler.scaler",
            "config": {"with_std": True, "with_mean": True, "copy": True}
        },
        # Keras model compile and fit
        "compile": {
            "loss": "categorical_crossentropy",
            "optimizer": tf.keras.optimizers.serialize(
                tf.keras.optimizers.Adam(learning_rate=0.001))
        },
        "fit": {
            "batch_size": 32, "epochs": 800, "verbose": 2, 
            "callbacks": []
        }
    }
})

#### Info

Some general information on the training, such as the used kgcnn version or a postfix for the output files.

In [5]:
hyper.update({
    "info":{ # Generla information
        "postfix": "_v1", # Appends _v1 to output folder
        "postfix_file": "_run2", # Appends _run2 to info files
        "kgcnn_version": "2.1.0"    
    }
})

## Benchmarks

Note that these are the results for models within `kgcnn` implementation, and that training is not always done with optimal hyperparameter or splits, when comparing with literature.
This table is generated automatically from keras history logs.
Model weights and training statistics plots are not uploaded on [github](https://github.com/aimat-lab/gcnn_keras/tree/master/training/results) due to their file size.

*Max.* or *Min.* denotes the best test error observed for any epoch during training.
To show overall best test error run ``python3 summary.py --min_max True``.
If not noted otherwise, we use a (fixed) random k-fold split for validation errors.

#### CoraLuDataset

Cora Dataset after Lu et al. (2003) of 2708 publications and 1433 sparse attributes and 7 node classes. Here we use random 5-fold cross-validation on nodes. 

| model     | kgcnn   |   epochs | Categorical accuracy   |
|:----------|:--------|---------:|:-----------------------|
| GAT       | 2.1.0   |      250 | 0.8490 &pm; 0.0122     |
| GATv2     | 2.1.0   |      250 | 0.8261 &pm; 0.0106     |
| GCN       | 2.1.0   |      300 | 0.8076 &pm; 0.0119     |
| GIN       | 2.1.0   |      500 | 0.8058 &pm; 0.0449     |
| GraphSAGE | 2.1.0   |      500 | **0.8512 &pm; 0.0100** |

####  CoraDataset

Cora Dataset of 19793 publications and 8710 sparse node attributes and 70 node classes. Here we use random 5-fold cross-validation on nodes. 

| model     | kgcnn   |   epochs | Categorical accuracy   |
|:----------|:--------|---------:|:-----------------------|
| GAT       | 2.1.0   |      250 | 0.6147 &pm; 0.0077     |
| GATv2     | 2.1.0   |     1000 | 0.6144 &pm; 0.0110     |
| GCN       | 2.1.0   |      300 | 0.6136 &pm; 0.0057     |
| GIN       | 2.1.0   |      800 | **0.6347 &pm; 0.0117** |
| GraphSAGE | 2.1.0   |      600 | 0.6133 &pm; 0.0045     |

####  ESOLDataset

ESOL consists of 1128 compounds as smiles and their corresponding water solubility in log10(mol/L). We use random 5-fold cross-validation. 

| model               | kgcnn   |   epochs | MAE [log mol/L]        | RMSE [log mol/L]       |
|:--------------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP         | 2.1.0   |      200 | 0.4562 &pm; 0.0084     | 0.6322 &pm; 0.0257     |
| CMPNN               | 2.1.0   |      600 | 0.4814 &pm; 0.0265     | 0.6821 &pm; 0.0193     |
| DimeNetPP           | 2.1.0   |      872 | 0.4576 &pm; 0.0422     | 0.6505 &pm; 0.0708     |
| DMPNN               | 2.1.0   |      300 | 0.4476 &pm; 0.0165     | 0.6349 &pm; 0.0152     |
| GAT                 | 2.1.0   |      500 | 0.4857 &pm; 0.0239     | 0.7028 &pm; 0.0356     |
| GATv2               | 2.1.0   |      500 | 0.4691 &pm; 0.0262     | 0.6724 &pm; 0.0348     |
| GCN                 | 2.1.0   |      800 | 0.5917 &pm; 0.0301     | 0.8118 &pm; 0.0465     |
| GIN                 | 2.1.0   |      300 | 0.5023 &pm; 0.0182     | 0.6997 &pm; 0.0300     |
| GIN.make_model_edge | 2.1.0   |      300 | 0.4881 &pm; 0.0173     | 0.6759 &pm; 0.0229     |
| GNNFilm             | 2.2.0   |      800 | 0.5145 &pm; 0.0158     | 0.7166 &pm; 0.0342     |
| GraphSAGE           | 2.1.0   |      500 | 0.5003 &pm; 0.0445     | 0.7242 &pm; 0.0791     |
| HamNet              | 2.1.0   |      400 | 0.5485 &pm; 0.0225     | 0.7605 &pm; 0.0210     |
| HDNNP2nd            | 2.2.0   |      500 | 0.7085 &pm; 0.0830     | 0.9806 &pm; 0.1386     |
| INorp               | 2.1.0   |      500 | 0.4856 &pm; 0.0145     | 0.6801 &pm; 0.0252     |
| MAT                 | 2.1.1   |      400 | 0.5341 &pm; 0.0263     | 0.7232 &pm; 0.0448     |
| MEGAN               | 2.2.1   |      400 | 0.4305 &pm; 0.0072     | 0.6073 &pm; 0.0186     |
| Megnet              | 2.1.0   |      800 | 0.5446 &pm; 0.0142     | 0.7651 &pm; 0.0410     |
| NMPN                | 2.1.0   |      800 | 0.5045 &pm; 0.0217     | 0.7092 &pm; 0.0482     |
| PAiNN               | 2.1.0   |      250 | **0.4291 &pm; 0.0164** | **0.6014 &pm; 0.0238** |
| RGCN                | 2.2.0   |      800 | 0.5014 &pm; 0.0274     | 0.7028 &pm; 0.0332     |
| Schnet              | 2.2.0   |      800 | 0.4664 &pm; 0.0188     | 0.6511 &pm; 0.0373     |

####  LipopDataset

Lipophilicity (MoleculeNet) consists of 4200 compounds as smiles. Graph labels for regression are octanol/water distribution coefficient (logD at pH 7.4). We use random 5-fold cross-validation. 

| model       | kgcnn   |   epochs | MAE [log mol/L]        | RMSE [log mol/L]       |
|:------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP | 2.1.0   |      200 | 0.4511 &pm; 0.0104     | 0.6193 &pm; 0.0149     |
| CMPNN       | 2.1.0   |      600 | 0.4129 &pm; 0.0069     | 0.5752 &pm; 0.0094     |
| DMPNN       | 2.1.0   |      300 | **0.3809 &pm; 0.0137** | **0.5503 &pm; 0.0251** |
| GAT         | 2.1.0   |      500 | 0.4954 &pm; 0.0172     | 0.6962 &pm; 0.0351     |
| GATv2       | 2.1.0   |      500 | 0.4081 &pm; 0.0099     | 0.5876 &pm; 0.0128     |
| GIN         | 2.1.0   |      300 | 0.4528 &pm; 0.0069     | 0.6382 &pm; 0.0286     |
| HamNet      | 2.1.0   |      400 | 0.4546 &pm; 0.0042     | 0.6293 &pm; 0.0139     |
| INorp       | 2.1.0   |      500 | 0.4635 &pm; 0.0106     | 0.6529 &pm; 0.0141     |
| MEGAN       | 2.1.0   |      400 | 0.3997 &pm; 0.0060     | 0.5635 &pm; 0.0114     |
| PAiNN       | 2.1.0   |      250 | 0.4033 &pm; 0.0123     | 0.5798 &pm; 0.0281     |
| Schnet      | 2.1.0   |      800 | 0.4788 &pm; 0.0046     | 0.6450 &pm; 0.0036     |

####  MatProjectEFormDataset

Materials Project dataset from Matbench with 132752 crystal structures and their corresponding formation energy in [eV/atom]. We use a random 5-fold cross-validation. 

| model                        | kgcnn   |   epochs | MAE [eV/atom]          | RMSE [eV/atom]         |
|:-----------------------------|:--------|---------:|:-----------------------|:-----------------------|
| CGCNN.make_crystal_model     | 2.1.1   |     1000 | 0.0369 &pm; 0.0003     | 0.0873 &pm; 0.0026     |
| DimeNetPP.make_crystal_model | 2.1.1   |      780 | 0.0233 &pm; 0.0005     | 0.0644 &pm; 0.0020     |
| MEGAN                        | 2.1.1   |      800 | 0.0397 &pm; 0.0009     | 0.0902 &pm; 0.0041     |
| Megnet.make_crystal_model    | 2.1.0   |     1000 | 0.0247 &pm; 0.0006     | 0.0639 &pm; 0.0028     |
| PAiNN.make_crystal_model     | 2.1.1   |      800 | 0.0244 &pm; 0.0002     | 0.0568 &pm; 0.0032     |
| Schnet.make_crystal_model    | 2.1.1   |      800 | **0.0215 &pm; 0.0003** | **0.0525 &pm; 0.0030** |

####  MatProjectPhononsDataset

Materials Project dataset from Matbench with 1,265 crystal structures and their corresponding vibration properties in [1/cm]. We use a random 5-fold cross-validation. 

| model                        | kgcnn   |   epochs | MAE [eV/atom]           | RMSE [eV/atom]          |
|:-----------------------------|:--------|---------:|:------------------------|:------------------------|
| CGCNN.make_crystal_model     | 2.1.1   |     1000 | 46.1204 &pm; 3.2640     | 106.4514 &pm; 16.9401   |
| DimeNetPP.make_crystal_model | 2.1.1   |      780 | 36.7288 &pm; 1.3484     | 81.5038 &pm; 10.3550    |
| MEGAN                        | 2.1.1   |      400 | 50.3682 &pm; 7.2162     | 121.6629 &pm; 27.4599   |
| Megnet.make_crystal_model    | 2.1.0   |     1000 | **29.2085 &pm; 2.8130** | **53.9366 &pm; 7.0800** |
| NMPN.make_crystal_model      | 2.1.0   |      700 | 44.4253 &pm; 3.7905     | 91.1708 &pm; 23.8014    |
| PAiNN.make_crystal_model     | 2.1.1   |      800 | 47.2212 &pm; 3.8855     | 82.7834 &pm; 6.0730     |
| Schnet.make_crystal_model    | 2.1.1   |      800 | 40.7416 &pm; 1.9105     | 80.2000 &pm; 10.1010    |

#### MutagenicityDataset

Mutagenicity dataset from TUDataset for classification with 4337 graphs. The dataset was cleaned for unconnected atoms. We use random 5-fold cross-validation. 

| model       | kgcnn   |   epochs | Accuracy               | AUC(ROC)               |
|:------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP | 2.1.0   |      200 | 0.7397 &pm; 0.0111     | 0.8207 &pm; 0.0111     |
| CMPNN       | 2.1.0   |      600 | 0.8102 &pm; 0.0157     | 0.8348 &pm; 0.0237     |
| DMPNN       | 2.1.0   |      300 | **0.8296 &pm; 0.0126** | 0.8714 &pm; 0.0075     |
| GAT         | 2.1.0   |      500 | 0.8008 &pm; 0.0115     | 0.8294 &pm; 0.0113     |
| GATv2       | 2.1.0   |      500 | 0.8029 &pm; 0.0122     | 0.8337 &pm; 0.0046     |
| GIN         | 2.1.0   |      300 | 0.8185 &pm; 0.0127     | **0.8734 &pm; 0.0094** |
| GraphSAGE   | 2.1.0   |      500 | 0.8165 &pm; 0.0061     | 0.8530 &pm; 0.0089     |
| INorp       | 2.1.0   |      500 | 0.7955 &pm; 0.0037     | 0.8255 &pm; 0.0047     |
| MEGAN       | 2.1.1   |      500 | 0.8137 &pm; 0.0117     | 0.8591 &pm; 0.0077     |

####  MUTAGDataset

MUTAG dataset from TUDataset for classification with 188 graphs. We use random 5-fold cross-validation. 

| model       | kgcnn   |   epochs | Accuracy               | AUC(ROC)               |
|:------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP | 2.1.0   |      200 | 0.8085 &pm; 0.1031     | 0.8471 &pm; 0.0890     |
| CMPNN       | 2.1.0   |      600 | 0.7873 &pm; 0.0724     | 0.7811 &pm; 0.0762     |
| DMPNN       | 2.1.0   |      300 | 0.8461 &pm; 0.0474     | 0.8686 &pm; 0.0480     |
| GAT         | 2.1.0   |      500 | 0.8351 &pm; 0.0920     | 0.8779 &pm; 0.0854     |
| GATv2       | 2.1.0   |      500 | 0.8144 &pm; 0.0757     | 0.8400 &pm; 0.0688     |
| GIN         | 2.1.0   |      300 | **0.8512 &pm; 0.0485** | **0.8861 &pm; 0.0922** |
| GraphSAGE   | 2.1.0   |      500 | 0.8193 &pm; 0.0445     | 0.8560 &pm; 0.0651     |
| INorp       | 2.1.0   |      500 | 0.8407 &pm; 0.0829     | 0.8549 &pm; 0.0705     |
| MEGAN       | 2.1.1   |      500 | 0.7977 &pm; 0.0663     | 0.8810 &pm; 0.0568     |

####  FreeSolvDataset

FreeSolv (MoleculeNet) consists of 642 compounds as smiles and their corresponding hydration free energy for small neutral molecules in water. We use a random 5-fold cross-validation. 

| model               | kgcnn   |   epochs | MAE [log mol/L]        | RMSE [log mol/L]       |
|:--------------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP         | 2.1.0   |      200 | 0.5853 &pm; 0.0519     | 1.0168 &pm; 0.1386     |
| CMPNN               | 2.1.0   |      600 | 0.5319 &pm; 0.0655     | 0.9262 &pm; 0.1597     |
| DimeNetPP           | 2.1.0   |      300 | 0.5791 &pm; 0.0649     | 0.9439 &pm; 0.1602     |
| DMPNN               | 2.1.0   |      300 | 0.5305 &pm; 0.0474     | **0.9070 &pm; 0.1497** |
| GAT                 | 2.1.0   |      500 | 0.5970 &pm; 0.0776     | 1.0107 &pm; 0.1554     |
| GATv2               | 2.1.0   |      500 | 0.6390 &pm; 0.0467     | 1.1203 &pm; 0.1491     |
| GCN                 | 2.1.0   |      800 | 0.7766 &pm; 0.0774     | 1.3245 &pm; 0.2008     |
| GIN                 | 2.1.0   |      300 | 0.7161 &pm; 0.0492     | 1.1171 &pm; 0.1233     |
| GIN.make_model_edge | 2.1.0   |      300 | 0.6285 &pm; 0.0588     | 1.0457 &pm; 0.1458     |
| GraphSAGE           | 2.1.0   |      500 | 0.5667 &pm; 0.0577     | 0.9861 &pm; 0.1328     |
| HamNet              | 2.1.0   |      400 | 0.6395 &pm; 0.0496     | 1.0508 &pm; 0.0827     |
| INorp               | 2.1.0   |      500 | 0.6448 &pm; 0.0607     | 1.0911 &pm; 0.1530     |
| MAT                 | 2.1.1   |      400 | 0.8477 &pm; 0.0488     | 1.2582 &pm; 0.0810     |
| MEGAN               | 2.1.1   |      400 | 0.5689 &pm; 0.0735     | 0.9689 &pm; 0.1602     |
| Megnet              | 2.1.0   |      800 | 0.9749 &pm; 0.0429     | 1.5328 &pm; 0.0862     |
| NMPN                | 2.1.0   |      800 | 0.5733 &pm; 0.0392     | 0.9861 &pm; 0.0816     |
| PAiNN               | 2.1.0   |      250 | **0.5128 &pm; 0.0565** | 0.9403 &pm; 0.1387     |
| Schnet              | 2.1.0   |      800 | 0.5980 &pm; 0.0556     | 1.0614 &pm; 0.1531     |

####  PROTEINSDataset

TUDataset of proteins that are classified as enzymes or non-enzymes. Nodes represent the amino acids of the protein. We use random 5-fold cross-validation. 

| model       | kgcnn   |   epochs | Accuracy               | AUC(ROC)               |
|:------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP | 2.1.0   |      200 | 0.7296 &pm; 0.0126     | 0.7967 &pm; 0.0264     |
| CMPNN       | 2.1.0   |      600 | 0.7377 &pm; 0.0164     | 0.7532 &pm; 0.0174     |
| DMPNN       | 2.1.0   |      300 | 0.7395 &pm; 0.0300     | **0.8038 &pm; 0.0365** |
| GAT         | 2.1.0   |      500 | 0.7314 &pm; 0.0283     | 0.7884 &pm; 0.0404     |
| GATv2       | 2.1.0   |      500 | 0.6999 &pm; 0.0266     | 0.7137 &pm; 0.0177     |
| GIN         | 2.1.0   |      150 | 0.7098 &pm; 0.0357     | 0.7437 &pm; 0.0454     |
| GraphSAGE   | 2.1.0   |      500 | 0.6937 &pm; 0.0273     | 0.7263 &pm; 0.0391     |
| INorp       | 2.1.0   |      500 | 0.7242 &pm; 0.0359     | 0.7333 &pm; 0.0228     |
| MEGAN       | 2.1.1   |      200 | **0.7449 &pm; 0.0222** | 0.8015 &pm; 0.0195     |

####  Tox21MolNetDataset

Tox21 (MoleculeNet) consists of 7831 compounds as smiles and 12 different targets relevant to drug toxicity. We use random 5-fold cross-validation. 

| model       | kgcnn   |   epochs | Accuracy               | AUC(ROC)               | BACC                   |
|:------------|:--------|---------:|:-----------------------|:-----------------------|:-----------------------|
| AttentiveFP | 2.2.1   |       50 | 0.9352 &pm; 0.0022     | 0.8127 &pm; 0.0100     | 0.6872 &pm; 0.0096     |
| CMPNN       | 2.2.1   |       30 | 0.9311 &pm; 0.0050     | 0.7769 &pm; 0.0344     | 0.6127 &pm; 0.0724     |
| DMPNN       | 2.2.1   |       50 | **0.9385 &pm; 0.0015** | 0.8295 &pm; 0.0103     | 0.6906 &pm; 0.0069     |
| GAT         | 2.2.1   |       50 | 0.9365 &pm; 0.0019     | 0.8309 &pm; 0.0053     | 0.6540 &pm; 0.0102     |
| GATv2       | 2.2.1   |       50 | 0.9366 &pm; 0.0019     | 0.8305 &pm; 0.0051     | 0.6775 &pm; 0.0093     |
| GIN         | 2.2.1   |       50 | 0.9358 &pm; 0.0031     | 0.8284 &pm; 0.0095     | 0.6986 &pm; 0.0129     |
| GraphSAGE   | 2.2.1   |      100 | 0.9286 &pm; 0.0042     | 0.8092 &pm; 0.0079     | **0.7056 &pm; 0.0087** |
| INorp       | 2.2.1   |       50 | 0.9335 &pm; 0.0032     | 0.8256 &pm; 0.0080     | 0.6854 &pm; 0.0119     |
| MEGAN       | 2.2.1   |       50 | 0.9374 &pm; 0.0033     | **0.8389 &pm; 0.0094** | 0.6627 &pm; 0.0133     |
| Schnet      | 2.2.1   |       50 | 0.9336 &pm; 0.0026     | 0.7856 &pm; 0.0054     | 0.6591 &pm; 0.0144     |

####  ClinToxDataset

ClinTox (MoleculeNet) consists of 1478 compounds as smiles and data of drugs approved by the FDA and those that have failed clinical trials for toxicity reasons. We use random 5-fold cross-validation. The first label 'approved' is chosen as target.

| model       | kgcnn   |   epochs | Accuracy               | AUC(ROC)               |
|:------------|:--------|---------:|:-----------------------|:-----------------------|
| AttentiveFP | 2.1.1   |       50 | 0.9372 &pm; 0.0095     | 0.8317 &pm; 0.0426     |
| CMPNN       | 2.1.1   |       30 | 0.9365 &pm; 0.0216     | 0.8067 &pm; 0.0670     |
| DMPNN       | 2.1.1   |       50 | 0.9385 &pm; 0.0146     | **0.8519 &pm; 0.0271** |
| GAT         | 2.1.1   |       50 | 0.9338 &pm; 0.0164     | 0.8354 &pm; 0.0487     |
| GATv2       | 2.1.1   |       50 | 0.9378 &pm; 0.0087     | 0.8331 &pm; 0.0663     |
| GIN         | 2.1.1   |       50 | 0.9277 &pm; 0.0139     | 0.8244 &pm; 0.0478     |
| GraphSAGE   | 2.1.1   |      100 | 0.9385 &pm; 0.0099     | 0.7795 &pm; 0.0744     |
| INorp       | 2.1.1   |       50 | 0.9304 &pm; 0.0106     | 0.7826 &pm; 0.0573     |
| MEGAN       | 2.1.1   |       50 | **0.9493 &pm; 0.0130** | 0.8394 &pm; 0.0608     |
| Schnet      | 2.1.1   |       50 | 0.9318 &pm; 0.0078     | 0.6807 &pm; 0.0745     |

####  QM7Dataset

QM7 dataset is a subset of GDB-13. Molecules of up to 23 atoms (including 7 heavy atoms C, N, O, and S), totalling 7165 molecules. We use dataset-specific 5-fold cross-validation. The atomization energies are given in kcal/mol and are ranging from -800 to -2000 kcal/mol). 

| model     | kgcnn   |   epochs | MAE [kcal/mol]         | RMSE [kcal/mol]        |
|:----------|:--------|---------:|:-----------------------|:-----------------------|
| DimeNetPP | 2.1.1   |      872 | 2.7266 &pm; 0.1022     | 6.1305 &pm; 0.9606     |
| EGNN      | 2.1.1   |      800 | 1.6182 &pm; 0.1712     | 3.8677 &pm; 0.7640     |
| HDNNP2nd  | 2.2.0   |      500 | 12.3555 &pm; 2.6972    | 25.6856 &pm; 11.3776   |
| MEGAN     | 2.1.1   |      800 | 10.4494 &pm; 1.6076    | 11.5596 &pm; 1.5710    |
| Megnet    | 2.1.1   |      800 | 1.4626 &pm; 0.0818     | 3.1522 &pm; 0.2409     |
| MXMNet    | 2.1.1   |      900 | **1.1078 &pm; 0.0799** | **2.8693 &pm; 0.7399** |
| NMPN      | 2.1.1   |      500 | 6.4698 &pm; 0.8256     | 35.0397 &pm; 4.3985    |
| PAiNN     | 2.1.1   |      872 | 1.2715 &pm; 0.0235     | 4.4958 &pm; 1.8048     |
| Schnet    | 2.1.1   |      800 | 2.5840 &pm; 0.3479     | 10.3788 &pm; 9.1047    |

####  QM9Dataset

QM9 dataset of 134k stable small organic molecules made up of C,H,O,N,F. Labels include geometric, energetic, electronic, and thermodynamic properties. We use a random 10-fold cross-validation, but not all splits are evaluated for cheaper evaluation. Test errors are MAE and for energies are given in [eV]. 

| model     | kgcnn   |   epochs | HOMO [eV]              | LUMO [eV]              | U0 [eV]                | H [eV]                 | G [eV]                 |
|:----------|:--------|---------:|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-----------------------|
| DimeNetPP | 2.1.0   |      600 | 0.0242 &pm; 0.0006     | 0.0209 &pm; 0.0002     | 0.0073 &pm; 0.0003     | **0.0073 &pm; 0.0003** | 0.0084 &pm; 0.0004     |
| EGNN      | 2.1.1   |      800 | 0.0273 &pm; 0.0004     | 0.0226 &pm; 0.0011     | 0.0081 &pm; 0.0002     | 0.0090 &pm; 0.0004     | 0.0095 &pm; 0.0005     |
| Megnet    | 2.1.0   |      800 | 0.0423 &pm; 0.0014     | 0.0354 &pm; 0.0008     | 0.0136 &pm; 0.0006     | 0.0135 &pm; 0.0001     | 0.0140 &pm; 0.0002     |
| MXMNet    | 2.1.1   |      900 | **0.0238 &pm; 0.0012** | **0.0203 &pm; 0.0007** | **0.0067 &pm; 0.0001** | 0.0074 &pm; 0.0008     | **0.0079 &pm; 0.0008** |
| NMPN      | 2.1.0   |      700 | 0.0627 &pm; 0.0013     | 0.0618 &pm; 0.0006     | 0.0385 &pm; 0.0011     | 0.0382 &pm; 0.0005     | 0.0365 &pm; 0.0005     |
| PAiNN     | 2.1.0   |      872 | 0.0287 &pm; 0.0068     | 0.0230 &pm; 0.0005     | 0.0075 &pm; 0.0002     | 0.0075 &pm; 0.0003     | 0.0087 &pm; 0.0002     |
| Schnet    | 2.1.0   |      800 | 0.0351 &pm; 0.0005     | 0.0293 &pm; 0.0006     | 0.0116 &pm; 0.0004     | 0.0117 &pm; 0.0004     | 0.0120 &pm; 0.0002     |

####  SIDERDataset

SIDER (MoleculeNet) consists of 1427 compounds as smiles and data for adverse drug reactions (ADR), grouped into 27 system organ classes. We use random 5-fold cross-validation.

| model     | kgcnn   |   epochs | Accuracy               | AUC(ROC)               |
|:----------|:--------|---------:|:-----------------------|:-----------------------|
| CMPNN     | 2.1.0   |       30 | 0.7360 &pm; 0.0048     | 0.5729 &pm; 0.0303     |
| DMPNN     | 2.1.0   |       50 | 0.6866 &pm; 0.1280     | 0.5942 &pm; 0.0508     |
| GAT       | 2.1.0   |       50 | 0.7559 &pm; 0.0078     | 0.6064 &pm; 0.0209     |
| GATv2     | 2.1.0   |       50 | 0.7515 &pm; 0.0066     | 0.6026 &pm; 0.0199     |
| GIN       | 2.1.0   |       50 | 0.7438 &pm; 0.0075     | 0.6109 &pm; 0.0256     |
| GraphSAGE | 2.1.0   |       30 | 0.7542 &pm; 0.0080     | 0.5946 &pm; 0.0151     |
| INorp     | 2.1.0   |       50 | 0.7471 &pm; 0.0105     | 0.5836 &pm; 0.0211     |
| MEGAN     | 2.1.1   |      150 | 0.7440 &pm; 0.0077     | **0.6186 &pm; 0.0160** |
| Schnet    | 2.1.0   |       50 | **0.7581 &pm; 0.0037** | 0.6075 &pm; 0.0143     |

####  MD17Dataset

Energies and forces for molecular dynamics trajectories of eight organic molecules. All geometries in A, energy labels in kcal/mol and force labels in kcal/mol/A. We use preset train-test split. Training on 1000 geometries, test on 500/1000 geometries. Errors are MAE for forces. Results are for the CCSD and CCSD(T) data in MD17. 

| model                      | kgcnn   |   epochs | Aspirin             | Toluene             | Malonaldehyde       | Benzene             | Ethanol             |
|:---------------------------|:--------|---------:|:--------------------|:--------------------|:--------------------|:--------------------|:--------------------|
| DimeNetPP.EnergyForceModel | 2.2.0   |     1000 | **0.5366 &pm; nan** | **0.2380 &pm; nan** | **0.3653 &pm; nan** | 0.0861 &pm; nan     | **0.2221 &pm; nan** |
| EGNN.EnergyForceModel      | 2.2.0   |     1000 | 2.4798 &pm; nan     | 0.8976 &pm; nan     | 0.9607 &pm; nan     | 0.3397 &pm; nan     | 0.5382 &pm; nan     |
| Megnet.EnergyForceModel    | 2.2.0   |     1000 | 2.2431 &pm; nan     | 1.0476 &pm; nan     | 1.7242 &pm; nan     | 0.5225 &pm; nan     | 1.4967 &pm; nan     |
| MXMNet.EnergyForceModel    | 2.2.0   |     1000 | 1.3700 &pm; nan     | 0.5998 &pm; nan     | 0.7752 &pm; nan     | 0.3669 &pm; nan     | 0.4451 &pm; nan     |
| NMPN.EnergyForceModel      | 2.2.0   |     1000 | 1.1429 &pm; nan     | 0.6937 &pm; nan     | 0.6134 &pm; nan     | 0.4112 &pm; nan     | 0.3220 &pm; nan     |
| PAiNN.EnergyForceModel     | 2.2.0   |     1000 | 0.8348 &pm; nan     | 0.2679 &pm; nan     | 0.7042 &pm; nan     | **0.0463 &pm; nan** | 0.5494 &pm; nan     |
| Schnet.EnergyForceModel    | 2.2.0   |     1000 | 1.0623 &pm; nan     | 0.5730 &pm; nan     | 0.6557 &pm; nan     | 0.2999 &pm; nan     | 0.4200 &pm; nan     |

####  MD17RevisedDataset

Energies and forces for molecular dynamics trajectories. All geometries in A, energy labels in kcal/mol and force labels in kcal/mol/A. We use preset train-test split. Training on 1000 geometries, test on 500/1000 geometries. Errors are MAE for forces. 

| model                      | kgcnn   |   epochs | Aspirin                | Toluene                | Malonaldehyde          | Benzene                | Ethanol                |
|:---------------------------|:--------|---------:|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:-----------------------|
| DimeNetPP.EnergyForceModel | 2.2.0   |     1000 | **0.5605 &pm; 0.0201** | **0.2207 &pm; 0.0117** | **0.4053 &pm; 0.0107** | 0.0656 &pm; 0.0055     | **0.2447 &pm; 0.0135** |
| EGNN.EnergyForceModel      | 2.2.0   |     1000 | 2.2138 &pm; 0.2762     | 0.8316 &pm; 0.0609     | 0.9882 &pm; 0.0411     | 0.3181 &pm; 0.0244     | 0.5367 &pm; 0.0386     |
| Megnet.EnergyForceModel    | 2.2.0   |     1000 | 2.3214 &pm; 0.2942     | 3.8695 &pm; 5.2614     | 1.6904 &pm; 0.1626     | 0.5341 &pm; 0.0907     | 1.2936 &pm; 0.0536     |
| MXMNet.EnergyForceModel    | 2.2.0   |     1000 | 1.8941 &pm; 0.0502     | 1.0880 &pm; 0.0628     | 1.2041 &pm; 0.0399     | 0.3573 &pm; 0.0302     | 0.6136 &pm; 0.0297     |
| NMPN.EnergyForceModel      | 2.2.0   |     1000 | 1.0653 &pm; 0.0263     | 0.6971 &pm; 0.0772     | 0.6197 &pm; 0.0327     | 0.3596 &pm; 0.0401     | 0.3444 &pm; 0.0219     |
| PAiNN.EnergyForceModel     | 2.2.0   |     1000 | 0.7802 &pm; 0.0124     | 0.2496 &pm; 0.0058     | 0.7427 &pm; 0.0116     | **0.0424 &pm; 0.0017** | 0.5627 &pm; 0.0236     |
| Schnet.EnergyForceModel    | 2.2.0   |     1000 | 0.9777 &pm; 0.0134     | 0.5378 &pm; 0.0100     | 0.6173 &pm; 0.0095     | 0.2537 &pm; 0.0034     | 0.4246 &pm; 0.0269     |

####  VgdMockDataset

Synthetic classification dataset containing 100 small, randomly generated graphs, where half of them were seeded with a triangular subgraph motif, which is the explanation ground truth for the target class distinction.

| model            | kgcnn   |   epochs | Categorical Accuracy   | Node AUC               | Edge AUC               |
|:-----------------|:--------|---------:|:-----------------------|:-----------------------|:-----------------------|
| GCN_GnnExplainer | 2.2.1   |      100 | 0.8700 &pm; 0.1122     | 0.7621 &pm; 0.0357     | 0.6051 &pm; 0.0416     |
| MEGAN            | 2.2.0   |      100 | **0.9400 &pm; 0.0490** | **0.8873 &pm; 0.0250** | **0.9518 &pm; 0.0241** |

#### VgdRbMotifsDataset

Synthetic graph regression dataset consisting of 5000 small, randomly generated graphs, where some of them are seeded with special red- or blue-dominated subgraph motifs, where blue motifs contribute negatively to a graph's overall target value and red motifs contribute positively. The explanation ground truth for this datasets consists of these motifs.

| model   | kgcnn   |   epochs | MSE                    | Node AUC               | Edge AUC               |
|:--------|:--------|---------:|:-----------------------|:-----------------------|:-----------------------|
| MEGAN   | 2.2.0   |      100 | **0.2075 &pm; 0.0421** | **0.9051 &pm; 0.0130** | **0.8096 &pm; 0.0414** |



> **NOTE**: You can find this page as jupyter notebook in https://github.com/aimat-lab/gcnn_keras/tree/master/docs/source